In [1]:
__author__ = "Lisette Espin"
__copyright__ = "Copyright 2019, GESIS / USC-ISI"
__credits__ = ["Lisette Espin-Noboa", 
               "Fariba Karimi", 
               "Bruno Ribeiro",
               "Kristina Lerman",
               "Claudia Wagner"]
__license__ = "GPL"
__version__ = "1.0.1"
__maintainer__ = "Lisette Espin"
__email__ = "lisette.espin@gesis.org"
__status__ = "Development"

<h1 style="text-align: center;">Example on New Synthetic Network</h1>

<h2>Dependencies</h2>

In [2]:
%load_ext autoreload
%autoreload 2

from org.gesis.network.network import Network
from org.gesis.sampling.sampling import Sampling
from org.gesis.local.local import Local
from org.gesis.relational.relational import Relational
from org.gesis.inference.inference import Inference
from utils import viz
from utils import io

<h2>Parameters</h2>

In [3]:
kind = "BAH"                    # network generative model type
N = 2000                        # number of nodes
m = 4                           # minimun degree (proxy of density)
H = 0.1                         # homophily
B = 0.1                         # class balance (minority fraction)
sampling = "nedges"              # subgraph sampling method (nodes, nedges, degree, neighbors, partial_crawls)
pseeds = 0.2                    # fraction of nodes to sample 
epoch = 1                       # epoch (iteration number #)
LC = "prior"                    # local model
RC = "nBC"                      # relational model
CI = "relaxation"               # inference
OUTPUT = "../results-individual"   # directory to store results

<h2>1. Network</h2>

In [4]:
net = Network(kind)
net.create_network(N=N,m=m,B=B,H=H)
net.info()

Name: homophilic_barabasi_albert
Type: Graph
Number of nodes: 2000
Number of edges: 7984
Average degree:   7.9840
{'attributes': ['color'], 'class': 'color', 'group': ['M', 'm'], 'labels': ['blue', 'red'], 'name': 'homophilic_barabasi_albert', 'kind': 'BAH', 'N': 2000, 'm': 4, 'B': 0.1, 'H': 0.1, 'i': 1, 'x': 1, 'n': 2000, 'e': 7984, 'h': 0.2, 'b': 0.1, 'min_degree': 4, 'k': 7.984, 'km': 32.605, 'kM': 5.248333333333333, 'fullname': 'BAH-N2000-m4-B0.1-H0.1-i1-x1-h0.2-k8.0-km32.6-kM5.2'}


<h2>2. Sampling</h2>

In [5]:
sam = Sampling(sampling, net.G, pseeds, epoch)
sam.extract_subgraph()
sam.info()

Name: homophilic_barabasi_albert
Type: Graph
Number of nodes: 400
Number of edges: 299
Average degree:   1.4950
{'attributes': ['color'], 'class': 'color', 'group': ['M', 'm'], 'labels': ['blue', 'red'], 'name': 'homophilic_barabasi_albert', 'kind': 'BAH', 'N': 400, 'm': 1, 'B': 0.17, 'H': 0.07263928750691771, 'i': 1, 'x': 1, 'e': 299, 'h': 0.2, 'k': 1.495, 'km': 3.4411764705882355, 'kM': 1.0963855421686748, 'fullname': 'BAH-N2000-m4-B0.1-H0.1-i1-x1-h0.2-k8.0-km32.6-kM5.2', 'pseeds': 0.2, 'method': 'nedges', 'epoch': 1}


<h2>3. Modeling</h2>

In [6]:
local_model = Local(LC)
local_model.learn(sam.Gseeds)
local_model.info()

blue    0.828358
red     0.171642
dtype: float64


In [7]:
relational_model = Relational(RC).get_model()
relational_model.learn(sam.Gseeds)
relational_model.info()

          blue       red
blue  0.368852  0.631148
red   0.978814  0.021186


<h2>4. Inference</h2>

In [ ]:
inference = Inference(CI)
inference.predict(net.G, local_model, relational_model)
inference.evaluation()
inference.summary()
inference.save(OUTPUT)

<h2>5. Results</h2>

In [ ]:
fn = None
viz.plot_rocauc_curve(inference.fpr, inference.tpr, inference.rocauc_curve, fn)